In [3]:
import pandas as pd

In [154]:
words_path = '/media/alex/nvme-a/vosk-model-ru-0.10/graph/words.txt'
words = pd.read_csv(words_path, sep=" ")
words.columns = ['line', 'id']
words[words.line.str.contains('шолохово')]

,line,id


### clean and concatenate streets

In [122]:
def street_data(path):
    
    with open(path, 'rt', encoding="utf8", errors='ignore') as file:
        streets = file.read()
    streets = streets.lower()
    
    correct = '\nйцукенгшщзхъфывапролджэячсмитьбю '
    used = ''.join(list(set(streets)))
    for u in used:
        if u not in correct:
            streets = streets.replace(u, ' ')
    while '  ' in streets:
        streets = streets.replace('  ', ' ')
    streets = streets.replace(' ', '\n')
    
    with open('b_'+path, 'w') as file:
        file.write(streets)
        
    streets = pd.read_csv('b_'+path, header=None)
    streets.columns = ['street']
    streets.street = pd.DataFrame(streets.street.unique())
    streets = streets.dropna()
    
    return streets

In [123]:
df = pd.DataFrame(columns = ['street'])
streets_path = ['streets_msk.csv', 'streets_spb.csv', 'streets_reg.csv']
for path in streets_path:
    df = pd.concat([df, street_data(path)], axis = 0)

In [134]:
df.street = pd.DataFrame(df.street.unique())
df = df.dropna()
df = pd.DataFrame(df[df.street.str.len()>3])
df.to_csv('all_streets.csv')

In [135]:
len(df)

35957

### missing streets

In [158]:
missing_words = df[df.street.isin(words.line)==False]
missing_words.to_csv('missing_streets.csv')
print('слов в модели:', len(words))
print('улиц в 1С:', len(df))
print('улиц, отсутствующих в модели:', len(missing_words))
missing_words.head()

слов в модели: 709156
улиц в 1С: 35957
улиц, отсутствующих в модели: 17108


,street
5,пыхчево
10,тигина
11,шестово
16,шолохово
24,верейское
